In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

## Day 48 Lecture 2 Assignment

In this assignment, we will apply density-based clustering to a dataset containing the locations of all Starbucks in the U.S.

This assignment will also use the haversine and plotly packages, which you should already have installed from the previous assignment.

In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN
import plotly.express as px

<IPython.core.display.Javascript object>

This dataset contains the latitude and longitude (as well as several other details we will not be using) of every Starbucks in the world as of February 2017. Each row consists of the following features, which are generally self-explanatory:

- Brand
- Store Number
- Store Name
- Ownership Type
- Street Address
- City
- State/Province
- Country
- Postcode
- Phone Number
- Timezone
- Longitude
- Latitude

Load in the dataset.

In [3]:
# answer goes here

# answer goes here

sbux = pd.read_csv('data/starbucks_locations.csv')

sbux.head()





,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude
0,Starbucks,47370-257954,"Meritxell, 96",Licensed,"Av. Meritxell, 96",Andorra la Vella,7,AD,AD500,376818720,GMT+1:00 Europe/Andorra,1.53,42.51
1,Starbucks,22331-212325,Ajman Drive Thru,Licensed,"1 Street 69, Al Jarf",Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.42
2,Starbucks,47089-256771,Dana Mall,Licensed,Sheikh Khalifa Bin Zayed St.,Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.39
3,Starbucks,22126-218024,Twofour 54,Licensed,Al Salam Street,Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.38,24.48
4,Starbucks,17127-178586,Al Ain Tower,Licensed,"Khaldiya Area, Abu Dhabi Island",Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.54,24.51


<IPython.core.display.Javascript object>

Begin by narrowing down the dataset to a specific geographic area of interest. Try just the United States; since you won't be calculating a distance matrix you can use more than just one state.

In [4]:
# answer goes here


# answer goes here
# sbux = sbux[sbux['State/Province']=='CA']





<IPython.core.display.Javascript object>

Build a DBSCAN clustering model using eps=2 (miles) and min_samples=5. Some tips that may be helpful:

1. Unlike our approach for hierarchical clustering, we do not need to calculate the NxN distance matrix for DBSCAN upfront. It directly supports the haversine distance metric, provided the nearest-neighbors algorithm is a ball tree. Set the "algorithm" and "metric" parameters to the appropriate values. 
2. Scikit-learn's implementation of haversine distance expects radians instead of degrees. Therefore, it would be advisable to create two new columns, Lat_Rad and Lon_Rad, that convert the Latitude and Longitude columns into radians. (Hint: there is a numpy function that does this.)  
3. The eps parameter, which corresponds to the radius of the neighborhood, will also need to be in radians. The conversion factor for miles to radians is approximately 1/3958.748; in other words, if you want the neighborhood to have a radius of 3 miles, set eps = 3/3958.748.  

Side note: ball-tree is an indexing structure that is very useful for nearest-neighbor calculations. The general time-complexity of finding a nearest neighbor using a Ball Tree is O(nlog(n)). This is a vast improvement over the naive O($n^{2}$) and allows us to cluster on much larger subsets of the data, like the entire country. Scikit-learn directly supports creating ball-trees through sklearn.neighbors.BallTree; if inclined, you could extend the analysis in the first after-lecture assignment (in which we calculated a similarity matrix for Hawaii) to the entire country using a BallTree and identify "island Starbucks locations" on a much larger scale.

Additionally, save the predicted cluster assignments as a new column in your dataframe.

In [5]:
import math

<IPython.core.display.Javascript object>

In [6]:
sbux = sbux.drop("Phone Number", 1)
sbux = sbux.dropna()

<IPython.core.display.Javascript object>

In [7]:
sbux["Lat_Rad"] = sbux["Latitude"].apply(lambda x: math.radians(x))

sbux["Lon_Rad"] = sbux["Longitude"].apply(lambda x: math.radians(x))

<IPython.core.display.Javascript object>

In [13]:
miles_conv = 1 / 3958.748
n = 25
eps = n * miles_conv

<IPython.core.display.Javascript object>

In [14]:
# answer goes here
clst = DBSCAN(eps=eps, metric='haversine', min_samples=2, algorithm='ball_tree')

clst.fit(sbux[['Lat_Rad', 'Lon_Rad']])


DBSCAN(algorithm='ball_tree', eps=0.006315127914178926, leaf_size=30,
       metric='haversine', metric_params=None, min_samples=2, n_jobs=None,
       p=None)

<IPython.core.display.Javascript object>

Finally, plot the resulting clusters on a map using the "scatter_geo" function from plotly.express. The map defaults to the entire world; the "scope" parameter is useful for narrowing down the region plotted in the map. The documentation can be found here:

https://www.plotly.express/plotly_express/#plotly_express.scatter_geo

How many clusters did DBSCAN produce? How many locations were treated as outliers (cluster = -1)?

In [15]:
# answer goes here

sbux["label"] = clst.labels_


sbux["string_label"] = "cluster " + sbux["label"].astype(str)

<IPython.core.display.Javascript object>

In [18]:
sbux["label"].value_counts()

443    2149
273    1609
425    1141
34      947
91      919
       ... 
35        2
333       2
293       2
309       2
404       2
Name: label, Length: 680, dtype: int64

<IPython.core.display.Javascript object>

In [16]:
# answer goes here

px.scatter_geo(data_frame=sbux, lat='Latitude', lon='Longitude', color='string_label', scope='usa',)



<IPython.core.display.Javascript object>

From the previous plot, we should see a very large number of clusters (400+). This would suggest that our definition of neighborhood may have been too strict. Experiment with other values of eps and min_samples and see how your changes affect the output. Output a map with what you think is the "best" clustering result below.

In [17]:
# answer goes here





<IPython.core.display.Javascript object>